In [1]:
import io
import os
import glob
import locale
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

/Users/edupmon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [3]:
pd.set_option('display.float_format',
              lambda x: locale.format_string('%.2f', x, grouping=True))

# Análise da exportação brasileira de vinhos de mesa

## Período de análise

In [4]:
ano_inicial = 2009
ano_final = 2023

## Obtendo os dados de exportação de vinhos

## Carregando os dados de exportação de vinhos em um dataframe

In [5]:
# Obtendo a lista de arquivos que será carregada no dataframe
arqs_csv = glob.glob(os.path.join('dados', 'exp-vinho-*.csv'))

# Definindo o dataframe que irá receber os dados de todos os arquivos CSV
'''Concatenação dos dataframes obtidos à partir da lista de arquivos (list
   comprehension abaixo), ignorando a primeira linha dos CSVs e definindo o
   nome das colunas manualmente'''
exportacao = pd.concat([pd.read_csv(arq,
                                    skiprows=1,
                                    na_values=0,
                                    names=['pais',
                                           'qtd_litros',
                                           'vlr_usd',
                                           'ano']) for arq in arqs_csv],
                       ignore_index=True)

exportacao['origem'] = 'Brasil'
exportacao['origem_iso_2'] = 'BR'
exportacao['origem_iso_3'] = 'BRA'

exportacao.fillna(0, inplace=True)
exportacao.head()

,pais,qtd_litros,vlr_usd,ano,origem,origem_iso_2,origem_iso_3
0,Afeganistão,"0,00","0,00",2021,Brasil,BR,BRA
1,África do Sul,"0,00","0,00",2021,Brasil,BR,BRA
2,"Alemanha, República Democrática","225.086,00","393.482,00",2021,Brasil,BR,BRA
3,Angola,"54.786,00","84.235,00",2021,Brasil,BR,BRA
4,Anguilla,"0,00","0,00",2021,Brasil,BR,BRA


## Analisando a exportação total por país

In [6]:
# Exportação total por país
total_por_pais = exportacao.groupby('pais')[['qtd_litros', 'vlr_usd']]\
                           .sum()\
                           .copy()

total_por_pais.head()

,qtd_litros,vlr_usd
pais,,
Afeganistão,"0,00","0,00"
"Alemanha, República Democrática","3.376.290,00","5.902.230,00"
Angola,"821.790,00","1.263.525,00"
Anguilla,"0,00","0,00"
Antilhas Holandesas,"123.525,00","159.765,00"


In [7]:
# Países sem exportação
paises_sem_exportacao = total_por_pais.query('qtd_litros == 0 & vlr_usd == 0')


'''Obtendo a lista de paises com exportação à partir dos conjutos de índices
   dos dois dataframes: total_por_pais - paises_sem_exportacao'''

conj_paises = set(total_por_pais.index.tolist())
conj_paises_sem_exportacao = set(paises_sem_exportacao.index.tolist())

conj_paises_com_exportacao = conj_paises - conj_paises_sem_exportacao


# Países com exportação
paises_com_exportacao = total_por_pais.loc[list(conj_paises_com_exportacao)]
# Adicionando a informação do valor por litro
paises_com_exportacao['usd_por_litro'] = paises_com_exportacao['vlr_usd']\
                                         / paises_com_exportacao['qtd_litros']

print(f'''
Período de análise.............: {ano_inicial} - {ano_final}
Total de países................: {len(conj_paises)} países
Total de países com exportação.: {len(conj_paises_com_exportacao)} países
''')


Período de análise.............: 2009 - 2023
Total de países................: 137 países
Total de países com exportação.: 39 países



## Doze países com maior volume total (em US$) das exportações

In [8]:
maiores_volumes = paises_com_exportacao.sort_values('vlr_usd',
                                                    ascending=False).head(12)

print(f'Dados do período total: {ano_inicial} - {ano_final}')
display(maiores_volumes)

Dados do período total: 2009 - 2023


,qtd_litros,vlr_usd,usd_por_litro
pais,,,
Rússia,"328.693.710,00","85.984.200,00","0,26"
Estados Unidos,"5.584.785,00","9.900.990,00","1,77"
China,"23.301.240,00","7.236.000,00","0,31"
"Alemanha, República Democrática","3.376.290,00","5.902.230,00","1,75"
Paraguai,"7.303.905,00","5.881.305,00","0,81"
Japão,"3.269.610,00","4.251.540,00","1,30"
Portugal,"2.115.000,00","2.533.845,00","1,20"
Países Baixos,"2.574.810,00","2.054.865,00","0,80"
Angola,"821.790,00","1.263.525,00","1,54"


O tabela acima demonstra, para o período total analisado os maiores volumes
exportados, bem como a variação do valor total em dólares e do valor, em dólar,
por litro para cada um dos países.

Após análise inicial dos dados, optou-se pelo foco nos países cujo volume de
exportação total no período analisado excedeu USD 1.000.000,00, pois, conforme
demonstrado na célula abaixo, estes correspondem a 99% do valor total exportado
em litros e 96% do valor total exportados em USD.

In [9]:
print('Valor total das exportações:')
display(paises_com_exportacao[['qtd_litros', 'vlr_usd']].sum())

print('\nValor total dos 12 países com maior valor nas exportações:')
display(maiores_volumes[['qtd_litros', 'vlr_usd']].sum())

print('\nParticipação dos 12 países no volume total de exportações:')
display(maiores_volumes[['qtd_litros', 'vlr_usd']].sum()\
        / paises_com_exportacao[['qtd_litros', 'vlr_usd']].sum())

Valor total das exportações:


qtd_litros   382.712.970,00
vlr_usd      134.114.820,00
dtype: float64


Valor total dos 12 países com maior valor nas exportações:


qtd_litros   378.142.275,00
vlr_usd      128.361.780,00
dtype: float64


Participação dos 12 países no volume total de exportações:


qtd_litros   0,99
vlr_usd      0,96
dtype: float64

## Enriquecimento dos dados

In [10]:
# Dataframe com dados apenas dos países que possuem exportações
exportacao_analise = exportacao[exportacao['pais']\
                     .isin(maiores_volumes.index.to_list())].copy()
exportacao_analise['usd_por_litro'] = exportacao_analise['vlr_usd']\
                                      / exportacao_analise['qtd_litros']

exportacao_analise.head()

,pais,qtd_litros,vlr_usd,ano,origem,origem_iso_2,origem_iso_3,usd_por_litro
2,"Alemanha, República Democrática","225.086,00","393.482,00",2021,Brasil,BR,BRA,"1,75"
3,Angola,"54.786,00","84.235,00",2021,Brasil,BR,BRA,"1,54"
26,Canadá,"15.664,00","73.445,00",2021,Brasil,BR,BRA,"4,69"
30,China,"1.553.416,00","482.400,00",2021,Brasil,BR,BRA,"0,31"
50,Estados Unidos,"372.319,00","660.066,00",2021,Brasil,BR,BRA,"1,77"


In [11]:
# Incluindo o código ISO dos países

pais_cod_iso = pd.read_csv('dados/pais-cod-iso.csv')
display(pais_cod_iso.head())

exportacao_analise = pd.merge(exportacao_analise, pais_cod_iso,
                              left_on='pais', right_on='pais',
                              how='left')
display(exportacao_analise.head())

,pais,pais_iso_2,pais_iso_3
0,"Alemanha, República Democrática",DE,DEU
1,Angola,AO,AGO
2,Canadá,CA,CAN
3,China,CN,CHN
4,Estados Unidos,US,USA


,pais,qtd_litros,vlr_usd,ano,origem,origem_iso_2,origem_iso_3,usd_por_litro,pais_iso_2,pais_iso_3
0,"Alemanha, República Democrática","225.086,00","393.482,00",2021,Brasil,BR,BRA,"1,75",DE,DEU
1,Angola,"54.786,00","84.235,00",2021,Brasil,BR,BRA,"1,54",AO,AGO
2,Canadá,"15.664,00","73.445,00",2021,Brasil,BR,BRA,"4,69",CA,CAN
3,China,"1.553.416,00","482.400,00",2021,Brasil,BR,BRA,"0,31",CN,CHN
4,Estados Unidos,"372.319,00","660.066,00",2021,Brasil,BR,BRA,"1,77",US,USA


In [12]:
# Manipulando os dados do Banco Mundial (databank.worldbank.org)

# Carregando os dados do arquivo CSV
banco_mundial = pd.read_csv('dados/databank-worldbank-org.csv', na_values='..')

# Excluindo as colunas que não são relevantes
banco_mundial.drop(['Country Name', 'Series Code'], axis=1, inplace=True)

# Renomeando as colunas
novo_nome = {'Country Code': 'pais_iso_3', 'Series Name': 'serie',
             '2009 [YR2009]': 2009, '2010 [YR2010]': 2010,
             '2011 [YR2011]': 2011, '2012 [YR2012]': 2012,
             '2013 [YR2013]': 2013, '2014 [YR2014]': 2014,
             '2015 [YR2015]': 2015, '2016 [YR2016]': 2016,
             '2017 [YR2017]': 2017, '2018 [YR2018]': 2018,
             '2019 [YR2019]': 2019, '2020 [YR2020]': 2020,
             '2021 [YR2021]': 2021, '2022 [YR2022]': 2022,
             '2023 [YR2023]': 2023}
banco_mundial.rename(columns=novo_nome, inplace=True)

# Trantando valor faltantes
intervalo = [i for i in range(ano_inicial, ano_final + 1)]

banco_mundial.loc[:, (intervalo)] = \
    banco_mundial.loc[:, (2009, 2010, 2011, 2012, 2013, 2014, 2015,
                          2016, 2017, 2018, 2019, 2020, 2021, 2022,
                          2023)].interpolate(method='linear',
                                             limit_direction='both', axis=1)

banco_mundial.head()

,pais_iso_3,serie,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AGO,GDP (current US$),"70.307.196.474,13","83.799.473.759,72","111.789.747.670,59","128.052.915.202,99","132.339.108.708,41","135.966.802.156,54","90.496.420.626,18","52.761.617.225,93","73.690.155.046,55","79.450.688.232,00","70.897.962.712,62","48.501.561.230,00","66.505.129.989,24","106.782.770.714,62","106.782.770.714,62"
1,AGO,GDP per capita (current US$),"3.123,70","3.586,66","4.608,16","5.083,83","5.061,35","5.011,98","3.217,34","1.809,71","2.439,37","2.540,51","2.191,35","1.450,91","1.927,47","3.000,44","3.000,44"
2,AGO,Total alcohol consumption per capita (liters o...,"8,32","8,32","8,21","8,10","7,98","7,87","7,76","7,36","6,97","6,57","6,17","6,17","6,17","6,17","6,17"
3,DEU,GDP (current US$),"3.411.261.212.652,34","3.399.667.820.000,01","3.749.314.991.050,59","3.527.143.188.785,16","3.733.804.649.549,03","3.889.093.051.023,52","3.357.585.719.351,56","3.469.853.463.945,53","3.690.849.152.517,65","3.974.443.355.019,60","3.889.177.589.254,90","3.887.727.161.914,41","4.278.503.934.689,85","4.082.469.490.797,68","4.082.469.490.797,68"
4,DEU,GDP per capita (current US$),"41.650,37","41.572,46","46.705,90","43.855,85","46.298,92","48.023,87","41.103,26","42.136,12","44.652,59","47.939,28","46.805,14","46.749,48","51.426,75","48.717,99","48.717,99"
